In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import tensorflow.keras
import warnings
warnings.filterwarnings("ignore")
from sklearn import metrics
import math
from math import log

In [15]:
data = pd.read_csv("noPH.csv")
#This is to separate different schools. UR, UB, VAN, MIT, STAN
filt_1 = data.iloc[0:144]
filt_2 = data.iloc[144:247]
filt_3 = data.iloc[247:404]
filt_4 = data.iloc[404:610]
filt_5 = data.iloc[610:752]

In [16]:
filt_5

,Unnamed: 0,research,theory,computer,science,engineering,systems,publication,areas,quantum,...,details,experiment,data,matter,award,software,graduate,studies,group,press
610,C:/Users/huang/Dropbox/AlexXuRichard/School te...,1.200269,0.525436,-0.000000,0.420178,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,...,-0.0,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,0.564045,0.227993,-0.000000,-0.000000
611,C:/Users/huang/Dropbox/AlexXuRichard/School te...,9.371353,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,19.410193,-0.000000,-0.0,...,-0.0,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,0.328497,0.132902,-0.000000,-0.000000
612,C:/Users/huang/Dropbox/AlexXuRichard/School te...,-0.302237,-0.000000,-0.000000,21.791269,-0.000000,-0.000000,197.571275,-0.000000,-0.0,...,-0.0,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,0.188803,1.133283,-0.000000,-0.000000
613,C:/Users/huang/Dropbox/AlexXuRichard/School te...,-1.648703,-0.000000,-1.344979,-0.000000,-0.000000,-0.000000,-0.000000,-0.482543,-0.0,...,-0.0,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
614,C:/Users/huang/Dropbox/AlexXuRichard/School te...,-4.477032,-0.000000,0.554766,-0.184588,-0.000000,0.035612,-0.732895,-0.000000,-0.0,...,-0.0,-0.0,-0.000000,-0.000000,-0.000000,0.432392,-0.000000,-0.000000,0.555962,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,C:/Users/huang/Dropbox/AlexXuRichard/School te...,-2.235934,-0.000000,-0.000000,2.229374,-0.000000,-0.299070,-0.285045,-0.256468,-0.0,...,-0.0,-0.0,-0.000000,-0.090072,1.182075,-0.000000,-0.000000,-0.000000,0.032187,-0.000000
748,C:/Users/huang/Dropbox/AlexXuRichard/School te...,-7.131232,-0.000000,-1.709340,-1.658931,-2.527222,-1.860021,-0.000000,-0.000000,-0.0,...,-0.0,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
749,C:/Users/huang/Dropbox/AlexXuRichard/School te...,-2.671176,-0.000000,-0.640552,-0.621657,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,...,-0.0,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.088463,-0.084441
750,C:/Users/huang/Dropbox/AlexXuRichard/School te...,-0.673552,-0.000000,0.263546,-0.139337,-0.146569,0.272030,-0.000000,-0.000000,-0.0,...,-0.0,-0.0,1.803123,-0.000000,-0.025392,-0.000000,-0.000000,-0.000000,-0.000000,0.011436


In [17]:
#The coefficient added in front of name is to separate different categories.
#Coefficient from 1000-5000 is in history category
#Coefficient from 6000-10000 is in physics category
#coefficient from 11000-15000 is in CS category

In [18]:
#For data in UR, the data <= 23 is history, data between 23 and 98 is physics, and data > 98 is CS
index_1 = []
name_1 = []
for name in filt_1['Unnamed: 0']:
    name_str_1 = str(name)
    name_str_1 = name_str_1.split('/')[6]
    name_str_school_1 = name_str_1.split(' ')[0]
    name_str_index_1 = name_str_1[14:]  
    name_str_index_1 = name_str_index_1[:-4]
    if int(name_str_index_1) <= 23:
        name_str_index_1 = str(1000 + int(name_str_index_1))
    elif int(name_str_index_1) <= 98:
        name_str_index_1 = str(6000 + int(name_str_index_1))
    else:
        name_str_index_1 = str(11000 + int(name_str_index_1))
    index_1.append(int(name_str_index_1))
    name_1.append(name_str_school_1 + name_str_index_1)
filt_1.index = name_1
filt_1 = filt_1.drop('Unnamed: 0', axis = 1)
filt_1['index'] = index_1
filt_1.sort_values(by = ['index'], inplace = True)
filt_1

,research,theory,computer,science,engineering,systems,publication,areas,quantum,view,...,experiment,data,matter,award,software,graduate,studies,group,press,index
UR1000,-0.000000,-0.00000,-0.000000,-0.000000,-1.651928,-0.0,-0.000000,-0.0,-0.0,-0.0,...,-0.0,-0.000000,-0.0,-0.000000,-0.0,-2.554438,-0.738531,-0.0,-0.000000,1000
UR1001,-2.573404,-0.00000,-0.000000,-0.000000,-0.440352,-0.0,1.316423,-0.0,-0.0,-0.0,...,-0.0,-0.000000,-0.0,17.144637,-0.0,-1.102714,-0.000000,-0.0,4.030341,1001
UR1002,-10.696343,-0.00000,-0.000000,-0.000000,-0.551570,-0.0,-0.110373,-0.0,-0.0,-0.0,...,-0.0,-0.594716,-0.0,-0.000000,-0.0,-1.404124,-0.124594,-0.0,-0.013771,1002
UR1003,-6.772392,-0.00000,-0.000000,-0.000000,-0.352838,-0.0,-0.068191,-0.0,-0.0,-0.0,...,-0.0,-0.000000,-0.0,-0.000000,-0.0,5.300289,-0.096421,-0.0,-0.000000,1003
UR1004,-0.000000,-0.00000,-0.000000,-0.000000,-2.456565,-0.0,-0.000000,-0.0,-0.0,-0.0,...,-0.0,-0.000000,-0.0,-0.000000,-0.0,-3.938366,-0.000000,-0.0,-0.000000,1004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UR11139,-4.157840,-1.61551,23.220628,1.154805,-1.050753,-0.0,-0.000000,-0.0,-0.0,-0.0,...,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.000000,-0.000000,-0.0,-0.000000,11139
UR11140,-0.000000,-0.00000,-0.000000,-4.008626,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.0,...,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.000000,-0.000000,-0.0,-0.000000,11140
UR11141,-0.000000,-0.00000,12.642411,-6.223224,-1.823512,-0.0,-0.000000,-0.0,-0.0,-0.0,...,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.000000,-0.000000,-0.0,-0.000000,11141
UR11142,-0.000000,-0.00000,12.661504,-6.209280,-1.818714,-0.0,-0.000000,-0.0,-0.0,-0.0,...,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.000000,-0.000000,-0.0,-0.000000,11142


In [19]:
#For data in UB, the data <= 25 is history, data between 25 and 63 is physics, and data > 63 is CS
index_2 = []
name_2 = []
for name in filt_2['Unnamed: 0']:
    name_str_2 = str(name)
    name_str_2 = name_str_2.split('/')[5]
    name_str_school_2 = name_str_2.split(' ')[0]
    name_str_index_2 = name_str_2[14:]
    name_str_index_2 = name_str_index_2[:-4]
    if int(name_str_index_2) <= 25:
        name_str_index_2 = str(2000 + int(name_str_index_2))
    elif int(name_str_index_2) <= 63:
        name_str_index_2 = str(7000 + int(name_str_index_2))
    else:
        name_str_index_2 = str(12000 + int(name_str_index_2))
    index_2.append(int(name_str_index_2))
    name_2.append(name_str_school_2 + name_str_index_2)
filt_2.index = name_2
filt_2 = filt_2.drop('Unnamed: 0', axis = 1)
filt_2['index'] = index_2
filt_2.sort_values(by = ['index'], inplace=True)
filt_2

,research,theory,computer,science,engineering,systems,publication,areas,quantum,view,...,experiment,data,matter,award,software,graduate,studies,group,press,index
UB2000,0.415512,-0.000000,-0.000000,3.164821,-0.000000,-0.000000,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.000000,-0.0,1.455123,-0.000000,17.692778,10.015214,-0.000000,6.237143,2000
UB2001,2.953502,-0.108898,-0.000000,0.414281,-0.000000,-0.000000,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.000000,-0.0,1.046266,-0.000000,1.751674,19.565521,-0.000000,38.690727,2001
UB2002,29.590293,-0.000000,0.121631,0.493697,0.137974,-0.000000,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.000000,-0.0,0.105269,-0.000000,12.089889,31.295803,-0.000000,5.036670,2002
UB2003,0.545853,-0.000000,-0.000000,3.118215,-0.000000,-0.000000,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.000000,-0.0,8.250367,-0.000000,10.470986,1.793996,-0.000000,-0.000000,2003
UB2004,7.852792,-0.000000,-0.000000,77.214774,-0.000000,-0.000000,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.000000,-0.0,0.063941,-0.000000,2.263782,4.996014,-0.000000,26.264635,2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UB12098,2.551737,-0.000000,77.879400,33.058901,76.913793,0.740595,-0.0,-0.0,-0.0,-0.0,...,-0.0,0.273177,-0.0,5.194189,-0.000000,0.742240,-0.000000,-0.000000,-0.000000,12098
UB12099,-0.754317,-0.000000,24.203631,19.322585,14.841627,0.622146,-0.0,-0.0,-0.0,-0.0,...,-0.0,0.274333,-0.0,-0.000000,-0.000000,0.744558,-0.000000,-0.000000,-0.000000,12099
UB12100,-1.822446,-0.000000,12.942431,8.651050,13.981157,-0.000000,-0.0,-0.0,-0.0,-0.0,...,-0.0,0.208228,-0.0,20.159797,-0.000000,0.391207,-0.000000,-0.000000,-0.000000,12100
UB12101,0.582402,-0.000000,23.218743,17.548472,24.596949,0.550521,-0.0,-0.0,-0.0,-0.0,...,-0.0,0.209390,-0.0,10.738502,1.089459,0.393567,-0.028431,-0.000000,-0.000000,12101


In [20]:
#For data in VAN, the data <= 41 is history, data between 41 and 74 is physics, and data > 74 is CS
index_3 = []
name_3 = []
for name in filt_3['Unnamed: 0']:
    name_str_3 = str(name)
    name_str_3 = name_str_3.split('/')[6] 
    name_str_school_3 = name_str_3.split(' ')[0]
    name_str_index_3 = name_str_3[15:]
    name_str_index_3 = name_str_index_3[:-4]
    if int(name_str_index_3) <= 41:
        name_str_index_3 = str(3000 + int(name_str_index_3))
    elif int(name_str_index_3) <= 74:
        name_str_index_3 = str(8000 + int(name_str_index_3))
    else:
        name_str_index_3 = str(13000 + int(name_str_index_3))
    index_3.append(int(name_str_index_3))
    name_3.append(name_str_school_3 + name_str_index_3)
filt_3.index = name_3
filt_3 = filt_3.drop('Unnamed: 0', axis = 1)
filt_3['index'] = index_3
filt_3.sort_values(by = ['index'], inplace=True)
filt_3

,research,theory,computer,science,engineering,systems,publication,areas,quantum,view,...,experiment,data,matter,award,software,graduate,studies,group,press,index
Van3000,-0.000000,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,-0.218646,-0.0,-0.000000,...,-0.0,-0.000000,-0.0,-0.000000,-0.000000,-0.207980,1.882219,-0.0,-0.000000,3000
Van3001,-3.364766,-0.0,-0.000000,-1.233460,-0.000000,-0.000000,-0.0,-0.258597,-0.0,-0.000000,...,-0.0,-0.000000,-0.0,1.232776,-0.000000,4.146437,10.872016,-0.0,9.201492,3001
Van3002,-0.000000,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,-0.118642,-0.0,-0.000000,...,-0.0,-0.000000,-0.0,-0.000000,-0.000000,0.681243,-0.051430,-0.0,-0.000000,3002
Van3003,-0.000000,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,-0.266145,-0.0,-0.000000,...,-0.0,-0.000000,-0.0,-0.000000,-0.000000,-0.107308,0.274274,-0.0,-0.000000,3003
Van3004,-1.881727,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,-0.149303,-0.0,-0.000000,...,-0.0,-0.000000,-0.0,-0.000000,-0.000000,-0.044933,0.259695,-0.0,1.149403,3004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Van13152,2.162496,-0.0,0.833125,3.992218,99.989789,2.439682,-0.0,-0.163366,-0.0,-0.020837,...,-0.0,0.190484,-0.0,5.696566,0.079224,0.098239,0.002783,-0.0,-0.000000,13152
Van13153,-6.790085,-0.0,10.643538,-1.155874,21.003100,0.434472,-0.0,-0.579502,-0.0,-0.073247,...,-0.0,10.327157,-0.0,-0.422066,-0.000000,-0.983932,-0.000000,-0.0,-0.000000,13153
Van13154,-6.412391,-0.0,2.562517,-0.985196,18.097450,-0.661324,-0.0,-0.498825,-0.0,-0.062472,...,-0.0,-0.564436,-0.0,-0.363066,-0.000000,-0.845727,-0.000000,-0.0,-0.000000,13154
Van13155,-5.359676,-0.0,6.028784,3.240499,33.033552,-0.552791,-0.0,-0.417466,-0.0,-0.051799,...,-0.0,-0.471690,-0.0,-0.303651,0.203763,-0.706780,-0.000000,-0.0,-0.000000,13155


In [21]:
#For data in MIT, the data <= 25 is history, data between 25 and 149 is physics, and data > 149 is CS
index_4 = []
name_4 = []
for name in filt_4['Unnamed: 0']:
    name_str_4 = str(name)
    name_str_4 = name_str_4.split('/')[6] 
    name_str_school_4 = name_str_4.split(' ')[0]
    name_str_index_4 = name_str_4[15:]
    name_str_index_4 = name_str_index_4[:-4]
    if int(name_str_index_4) <= 25:
        name_str_index_4 = str(4000 + int(name_str_index_4))
    elif int(name_str_index_4) <= 149:
        name_str_index_4 = str(9000 + int(name_str_index_4))
    else:
        name_str_index_4 = str(14000 + int(name_str_index_4))
    index_4.append(int(name_str_index_4))
    name_4.append(name_str_school_4 + name_str_index_4)
filt_4.index = name_4
filt_4 = filt_4.drop('Unnamed: 0', axis = 1)
filt_4['index'] = index_4
filt_4.sort_values(by = ['index'], inplace=True)
filt_4

,research,theory,computer,science,engineering,systems,publication,areas,quantum,view,...,experiment,data,matter,award,software,graduate,studies,group,press,index
MIT4000,2.422834,-0.000000,-0.000000,4.481273,-0.000000,-0.000000,-0.0,-0.000000,-0.0,-0.0,...,-0.000000,-0.000000,-0.0,1.767325,-0.000000,19.409449,10.445144,-0.000000,6.525958,4000
MIT4001,5.097242,-0.066289,-0.000000,1.113265,-0.000000,-0.000000,-0.0,-0.000000,-0.0,-0.0,...,-0.000000,-0.000000,-0.0,1.285374,-0.000000,2.340007,20.088123,-0.000000,39.304124,4001
MIT4002,41.524599,-0.000000,0.838225,2.830420,0.855825,-0.000000,-0.0,-0.000000,-0.0,-0.0,...,-0.000000,-0.000000,-0.0,0.460957,-0.000000,14.982322,32.526002,-0.000000,5.524191,4002
MIT4003,3.401371,-0.000000,-0.000000,5.026226,-0.000000,-0.000000,-0.0,-0.000000,-0.0,-0.0,...,-0.000000,-0.000000,-0.0,9.090353,-0.000000,12.256154,2.025689,-0.000000,-0.000000,4003
MIT4004,12.115495,-0.000000,-0.000000,83.934198,-0.000000,-0.000000,-0.0,-0.000000,-0.0,-0.0,...,-0.000000,-0.000000,-0.0,0.233261,-0.000000,3.180536,5.351339,-0.000000,26.983836,4004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MIT14201,-1.705710,-1.718474,-0.879812,-1.741134,-0.000000,-0.000000,-0.0,-0.000000,-0.0,-0.0,...,-0.000000,-0.395748,-0.0,-0.349029,-0.128120,-0.000000,-0.000000,-0.000000,-0.000000,14201
MIT14202,-0.000000,-0.000000,-6.275043,-11.441829,-6.104522,-0.000000,-0.0,-0.000000,-0.0,-0.0,...,-0.000000,-0.000000,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,14202
MIT14203,-1.324831,-0.000000,5.483281,-0.077324,-0.186140,-0.000000,-0.0,-0.153454,-0.0,-0.0,...,-0.104815,-0.105886,-0.0,16.807332,-0.000000,-0.000000,-0.000000,-0.003611,-0.000000,14203
MIT14204,0.251618,-0.084475,20.827231,1.448197,0.486219,105.708526,-0.0,-0.000000,-0.0,-0.0,...,-0.000000,4.464344,-0.0,1.216309,2.500269,-0.000000,-0.000000,-0.000000,-0.007330,14204


In [22]:
#For data in STAN, the data <= 43 is history, data between 43 and 57 is physics, and data > 57 is CS
index_5 = []
name_5 = []
for name in filt_5['Unnamed: 0']:
    name_str_5 = str(name)
    name_str_5 = name_str_5.split('/')[6] 
    name_str_school_5 = name_str_5.split(' ')[0]
    name_str_index_5 = name_str_5[16:]
    name_str_index_5 = name_str_index_5[:-4]
    if int(name_str_index_5) <= 43:
        name_str_index_5 = str(5000 + int(name_str_index_5))
    elif int(name_str_index_5) <= 57:
        name_str_index_5 = str(10000 + int(name_str_index_5))
    else:
        name_str_index_5 = str(15000 + int(name_str_index_5))
    index_5.append(int(name_str_index_5))
    name_5.append(name_str_school_5 + name_str_index_5)
filt_5.index = name_5
filt_5 = filt_5.drop('Unnamed: 0', axis = 1)
filt_5['index'] = index_5
filt_5.sort_values(by = ['index'], inplace=True)
filt_5

,research,theory,computer,science,engineering,systems,publication,areas,quantum,view,...,experiment,data,matter,award,software,graduate,studies,group,press,index
Stan5000,1.200269,0.525436,-0.000000,0.420178,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,-0.000000,...,-0.0,-0.000000,-0.0,-0.000000,-0.0,0.564045,0.227993,-0.000000,-0.000000,5000
Stan5001,9.371353,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,19.410193,-0.000000,-0.0,-0.000000,...,-0.0,-0.000000,-0.0,-0.000000,-0.0,0.328497,0.132902,-0.000000,-0.000000,5001
Stan5002,8.579983,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,48.179511,1.053868,-0.0,-0.000000,...,-0.0,2.623006,-0.0,-0.000000,-0.0,14.346019,0.048401,-0.000000,2.954302,5002
Stan5003,15.951008,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,76.949752,-0.000000,-0.0,-0.000000,...,-0.0,-0.000000,-0.0,-0.000000,-0.0,0.334664,8.311087,-0.000000,14.454761,5003
Stan5004,3.723611,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,30.560286,-0.000000,-0.0,-0.000000,...,-0.0,-0.000000,-0.0,-0.000000,-0.0,0.312593,0.153957,-0.000000,-0.000000,5004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Stan15137,1.969160,-0.000000,4.420371,49.879686,11.453568,-0.000000,-0.000000,-0.000000,-0.0,-0.000000,...,-0.0,22.871539,-0.0,12.763654,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,15137
Stan15138,0.032520,-0.000000,-0.000000,-0.023703,-0.000000,5.596857,-0.000000,-0.000000,-0.0,-0.000000,...,-0.0,19.137306,-0.0,-0.000000,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,15138
Stan15139,6.342011,-0.000000,-0.023687,0.395851,-0.000000,3.866156,-0.000000,-0.000000,-0.0,-0.030405,...,-0.0,0.172494,-0.0,0.025274,-0.0,0.000858,-0.000000,-0.000000,-0.000000,15139
Stan15140,-1.409676,-0.000000,24.502687,0.732851,2.227217,-0.000000,-0.000000,-0.000000,-0.0,-0.000000,...,-0.0,-0.000000,-0.0,5.716478,-0.0,-0.000000,-0.000000,0.007511,-0.000000,15140


In [23]:
data_ult = pd.merge(filt_1, filt_2, how = 'outer')
data_ult = pd.merge(data_ult, filt_3, how = 'outer')
data_ult = pd.merge(data_ult, filt_4, how = 'outer')
data_ult = pd.merge(data_ult, filt_5, how = 'outer')
data_ult.sort_values(by = ['index'], inplace = True)
data_ult = data_ult.drop('index', axis = 1)
print(data_ult)
data_ult.to_csv('data_ult.csv')

      research  theory   computer    science  engineering   systems  \
0    -0.000000    -0.0  -0.000000  -0.000000    -1.651928 -0.000000   
1    -2.573404    -0.0  -0.000000  -0.000000    -0.440352 -0.000000   
2   -10.696343    -0.0  -0.000000  -0.000000    -0.551570 -0.000000   
3    -6.772392    -0.0  -0.000000  -0.000000    -0.352838 -0.000000   
4    -0.000000    -0.0  -0.000000  -0.000000    -2.456565 -0.000000   
..         ...     ...        ...        ...          ...       ...   
747   1.969160    -0.0   4.420371  49.879686    11.453568 -0.000000   
748   0.032520    -0.0  -0.000000  -0.023703    -0.000000  5.596857   
749   6.342011    -0.0  -0.023687   0.395851    -0.000000  3.866156   
750  -1.409676    -0.0  24.502687   0.732851     2.227217 -0.000000   
751  -1.887824    -0.0  -0.550843  -0.744606    -0.000000 -0.000000   

     publication  areas  quantum      view   details  experiment       data  \
0      -0.000000   -0.0     -0.0 -0.000000 -0.000000        -0.0  -0